In [19]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from IPython.display import display, HTML

# Load dataset
df = pd.read_csv(r'/Users/nishthatiwari/Desktop/cognifyz/RR/Dataset .csv')


In [20]:
# Handle missing values
df.fillna('', inplace=True)

# One-hot encode categorical variables
categorical_cols = ['Cuisines', 'Has Table booking', 'Has Online delivery', 'Is delivering now', 'Switch to order menu']
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = onehot_encoder.fit_transform(df[categorical_cols])

# Combine encoded features with numerical features
features = pd.concat([df[['Price range', 'Aggregate rating']], pd.DataFrame(encoded_features.toarray())], axis=1)


In [21]:
# Function to recommend restaurants based on user preferences
def recommend_restaurants(user_preferences, df, features, top_n=5):
    # Encode user preferences
    user_pref_df = pd.DataFrame([user_preferences])
    user_pref_encoded = onehot_encoder.transform(user_pref_df[categorical_cols])
    user_features = pd.concat([user_pref_df[['Price range', 'Aggregate rating']], pd.DataFrame(user_pref_encoded.toarray())], axis=1)

    # Compute similarity
    similarity = cosine_similarity(user_features, features)

    # Get top N recommendations
    top_indices = similarity[0].argsort()[-top_n:][::-1]
    recommendations = df.iloc[top_indices]

    return recommendations


In [22]:
# Example user preferences
user_preferences = {
    'Cuisines': 'Italian',
    'Has Table booking': 'Yes',
    'Has Online delivery': 'No',
    'Is delivering now': 'No',
    'Switch to order menu': 'No',
    'Price range': 2,
    'Aggregate rating': 4.0
}

# Get recommendations
recommendations = recommend_restaurants(user_preferences, df, features)
# Display recommendations as a table
table = tabulate(recommendations[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']], headers='keys', tablefmt='html')
display(HTML(table))

,Restaurant Name,Cuisines,Price range,Aggregate rating
3733,Amalfi,Italian,3,3.9
1231,Italiano,Italian,3,3.6
9083,Ristorante Prego,Italian,3,3.1
9195,Coffee Cup,"Cafe, Continental",2,4.6
9162,Kargo,"Charcoal Grill, Italian, North Indian, European, Indonesian, Thai",2,4.4
